In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt
import seaborn as sns

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
df = pd.read_csv("/kaggle/input/phishing-url-detection/out.csv")
df = df.sample(frac=1, random_state=42).reset_index(drop=True)          #Kullandığım veri seti zararlı 
#ve zararsız label değerlerine göre sıralandığı için ve boyutunun büyük olması nedeniyle
#sınırlamam gerekmesiyle bu yöntemi kullanrak satırları ayrıca karıştırdım.
df = df.head(30000)       #İlk 30000 satırı dataframe'e ekledim.

In [ ]:
df.describe()       #Veri seti hakkında bilgileri yazdırdım.

In [ ]:
df.head()       #İlk 5 satır veriyi inceledim. Hem zararlı hem de zararsız URL'leri de içeriyor.

In [ ]:
df.info()       #Hücrelerde boş kısım var mı kontrol ettim. Yok gibi görünüyordu ancak fit fonksiyonu
#çalışınca yine de hata veriyordu.

In [ ]:
df.isnull().sum()       #Sütunlara göre Null veri sayılarını yazan bu fonksiyonları çalıştırdım ve
#hois_data ve domain_age_days adlı sütunlarda Null değvrler olduğunu tespit ettim.

In [ ]:
df['label'].value_counts()       #Ayrıca label adlı sütundaki değerlerin sayılarının istenmeyecek
#kadar çok yakın olduğunu gördüm.

In [ ]:
df = df.dropna(subset=['domain_age_days', 'whois_data'])       #Bu iki sütunu tablodan sildim.
df.isnull().sum()       #Tekrar Null içeren sütunları görmek için bu fonksiyonu çalıştırdım ve
#Null öğe kalmamıştı.

In [ ]:
df['label'].value_counts()       #Tekrar label sütununun veri değerleri sayılarının uzaklığı gerçeğe daha
#yakın sonuç alabilmemiz için yeterli görünüyor. Buraya kadar eda işlemleri tamamlanmış oldu.

In [ ]:
plt.figure(figsize=(10, 6))
sns.countplot(x='label', data=df)
plt.title('Sınıf Dağılımı')
plt.xlabel('Sınıf')
plt.ylabel('Sayı')
plt.xticks(ticks=[0, 1], labels=['Phishing', 'Benign'])  # Etiketleri güncelle
plt.show()

In [ ]:
plt.figure(figsize=(12, 6))
sns.barplot(x='source', y='label', data=df, estimator=lambda x: len(x) / len(df) * 100)  # Yüzde olarak gösterim
plt.title('Kaynaklara Göre Sınıf Dağılımı')
plt.xlabel('Kaynak')
plt.ylabel('Yüzde')
plt.xticks(rotation=45)
plt.show()

In [ ]:
#Denetimli Öğrenme Modelleri

from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier, AdaBoostClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.neural_network import MLPClassifier
from sklearn.linear_model import LogisticRegression 
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix

#Buraya kadar denetimli öğrenme modellerini ve çeşitli sınıflandırma ve etiketleme modüllerini import ettim

label_encoder = LabelEncoder()
df['label'] = label_encoder.fit_transform(df['label'])
df['source'] = label_encoder.fit_transform(df['source'])
#Bazı sütunlarım (label ve source) kategorik veriler içeriyordu, bunları Label encoder modülü ile etiketledim

def denetimli_modeller(models):
    #Denetmli modelleri tek tek veri seti üzerinde deneyip accuracy scoure değerine göre en kullanışlı olanı
    #belirleyecek bir fonksiyon yazdım.
    X = df.drop(columns=['url', 'label', 'whois_data'])       #Denetimli öğrenmede hedef çıktımız label sütununda 
    #bulunduğu için label sütunu, url sütunundaki veriler ise hem işe yaramaz hem de karmaşık olduğundan 
    #bu iki sütunu drop ediyoruz. Kalan sütunları da kaynak dataframe'imiz olacak X'e atıyoruz.
    y = df['label']       #Hedefimizdeki sütunu ise y adlı dataframe'e atıyoruz karşılaştırmak için.
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)
    #Burada ise eğitim ve test kümelerimizi oluşturuyoruz. Verilvrimizin %30'unu teste ayırıyor ve
    #daha etkili bir test oluşturmak için 42 parametresine göre verileri karıştırıyoruz.
    
    scaler = StandardScaler()
    X_train_scaled = scaler.fit_transform(X_train)
    X_test_scaled = scaler.transform(X_test)
    #StandartScaler modülü ile veri normalizasyonu (güzelleme) yapıyoruz.
    

    best_model_name = None
    best_accuracy = 0
    
    for name, model in models.items():
        model.fit(X_train_scaled, y_train)       #fit fonksiyonu ile makinemize eğitilecek kümeleri veriyoruz
        #ve eğitmeye başlıyoruz sırayla model adlı dizide bulunan modellerle.
        y_pred = model.predict(X_test_scaled)       #Sonucu normalize edilmiş test verimiz ile karşılaştırıyoruz.
        
        accuracy = accuracy_score(y_test, y_pred)       #Modellerin doğruluk değerleri değişkene atanır.
        precision = precision_score(y_test, y_pred)
        recall = recall_score(y_test, y_pred)
        f1 = f1_score(y_test, y_pred)
        
        print(f"Model: {name}")       #Sırayla her model karışıklık matrisi ile çıktıya yazılır.
        print(classification_report(y_test, y_pred))
        print(confusion_matrix(y_test, y_pred))
        
        """Cross Validation işlemi, bu kısmı tam olarak çalışır hale getiremedim :')
        scores = cross_val_score(best_model_name, X_scaled, y, cv=5, scoring='accuracy')
        mean_accuracy = np.mean(scores)
        print(f"Model: {model_name}, Cross-Validation Mean Accuracy: {mean_accuracy:}")"""
        
        if accuracy > best_accuracy:
            best_accuracy = accuracy
            best_model_name = name
        #Bu if bloğu içinde ise en yüksek doğruluk değerine sahip model belirlenir.
    
    return best_model_name, best_accuracy       #en yüksek doğruluk değerine sahip model ve değeri 
    #return edilir.


models = {
    'Logistic Regression': LogisticRegression(max_iter=200),
    'SVM': SVC(kernel='linear'),
    'Decision Tree': DecisionTreeClassifier(),
    'Random Forest': RandomForestClassifier(n_estimators=100, random_state=42),
    'Gradient Boosting': GradientBoostingClassifier(n_estimators=100, random_state=42),
    'AdaBoost': AdaBoostClassifier(n_estimators=50, random_state=42),
    '.Naive Bayes': GaussianNB(),
    'Neural Network': MLPClassifier(hidden_layer_sizes=(100,), max_iter=500, random_state=42),
    'K-Nearest Neighbors': KNeighborsClassifier(n_neighbors=5)
}       #Denetimli öğrenme modellerini tutan küme, fonksiyona göndermek üzere.

best_model_name, best_accuracy = denetimli_modeller(models)       #Fonksiyon çalıştırılır.
print(f"\nBest Model: {best_model_name} with Accuracy: {best_accuracy:}")
#en iyi sonuç veren model ve doğruluk değeri yazdırılır.

In [ ]:
#Denetimsiz Öğrenme Modelleri

from sklearn.cluster import KMeans, AgglomerativeClustering, DBSCAN
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.metrics import silhouette_score
from sklearn.model_selection import train_test_split
import pandas as pd
#Kullanacağımız modüllerin import edilmesi

label_encoder = LabelEncoder()
df['source'] = label_encoder.fit_transform(df['source'])
#Kategorik veriler içeren sütun (source) encode edilmesi

X = df.drop(columns=['url', 'label', 'whois_data'])       #Kullanmayacağımız sütunların dataframe'den çıkarılması

scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)
#Verilerin normalize edilmesi

models = {
    'KMeans': KMeans(n_clusters=2, random_state=42, n_init = 10),
    'Agglomerative Clustering': AgglomerativeClustering(n_clusters=2),
    'DBSCAN': DBSCAN(eps=5, min_samples=5)
}       #Fonksiyona gönderilmek üzere denetimsiz öğrenme modellerinin kümede tutulması

def denetimsiz_modeller(models, X_scaled):
    best_model_name = None
    best_score = -1
    
    #Bu döngü ile modellerin fit_predict mi yoksa fit fonksiyonunu mu kullandığının tespitine göre öğrenme
    #işlemi başlatılır.
    for name, model in models.items():
        if hasattr(model, 'fit_predict'):
            labels = model.fit_predict(X_scaled)
        else:
            model.fit(X_scaled)
            labels = model.labels_
        
        # Silhouette score hesaplama, denetimli öğrenmedeki gibi doğruluk oranının karşılığı olarak açıklanıyor
        if len(set(labels)) > 1:  # Tek bir küme olursa silhouette score hesaplanamayacağından bunun kontrolü yapılıyor.
            score = silhouette_score(X_scaled, labels)
            print(f"Model: {name}, Silhouette Score: {score:.4f}")
        
            # En iyi sonuca göre en iyi denetimsiz öğrenme modelinin belirlenmesi ve en son da return edilmesi
            if score > best_score:
                best_score = score
                best_model_name = name
    
    return best_model_name, best_score

# En iyi modeli bulma
best_model_name, best_score = denetimsiz_modeller(models, X_scaled)       #Denetimsiz modeller fonksiyonunu çalıştırmaya yarar.
print(f"Best Model: {best_model_name} with Silhouette Score: {best_score:.4f}")
#Belirlenen en iyi denetimsiz eğitim modeli ve silhouette değeri yazılır.

In [ ]:
#Son olarak şu kazanım; DBSCAN modelinde eps değerini arttırdıkça silhouette
#değerinin de arttırğını gördüm.